<a href="https://colab.research.google.com/github/MocT117/datasciencecoursera/blob/master/FollowupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pandas as pd
from datetime import datetime
import warnings

El siguiente código tiene como propósito generar un archivo excel que se actualice diariemente con la información de la cadena de suminstro de krones



In [5]:
from google.colab import files
import io
uploaded = files.upload() #archivos que se van a procesar
#conversion de los archivos a dtf
dataframes = {}

for filename in uploaded.keys():
  try:
    df = pd.read_excel(io.BytesIO(uploaded[filename]), engine="openpyxl")
    dataframes[filename] = df
    print(f"Arhivo cargado: {filename} | Filas: {df.shape[0]} | Columnas: {df.shape[1]}" )
  except Exception as e:
    print(f"Error al leer {filename}: {e}")



Saving CONTROL DE IMPORTACIONES.xlsx to CONTROL DE IMPORTACIONES.xlsx
Saving EXW KAG.XLSX to EXW KAG.XLSX
Saving Follow-up SPS DC03.XLSX to Follow-up SPS DC03.XLSX
Saving SO KAG.XLSX to SO KAG.XLSX
Arhivo cargado: CONTROL DE IMPORTACIONES.xlsx | Filas: 4495 | Columnas: 21
Arhivo cargado: EXW KAG.XLSX | Filas: 7311 | Columnas: 36
Arhivo cargado: Follow-up SPS DC03.XLSX | Filas: 15884 | Columnas: 32
Arhivo cargado: SO KAG.XLSX | Filas: 18485 | Columnas: 11


In [18]:
df_base = "Follow-up SPS DC03.XLSX"
if df_base in dataframes:
  df_base = dataframes[df_base]
  print("Columnas del DataFrame base:")
  print("df_base.colums.tolist()")
else:
        print(f"No se encontró el archivo {df_base}en los archivos cargados")



Columnas del DataFrame base:
df_base.colums.tolist()


Eliminación de columnas

In [45]:
columnas_a_eliminar = ["Centro de coste","Elemento PEP", "Orden", "Activo fijo","Subnúmero", "Grafo","Operación", "Indicador de borrado","Imputación actual",'Cl.documento compras','Tipo doc.compras','Cantidad base','Cantidad de posiciones','Grupo de compras']
for col in columnas_a_eliminar:
  if col not in df_base.columns:
    print(f" La columna '{col}' no está en el df y será ignorada.")
df_base_limpio = df_base.drop(columns=[col for col in columnas_a_eliminar if col in df_base.columns])
print("\Columnas despues de eliminar las especificaciones:")
print(df_base_limpio.columns.tolist())

\Columnas despues de eliminar las especificaciones:
['Documento compras', 'Posición', 'Doc.comercial', 'Posición.1', 'Fecha documento', 'Proveedor/Centro suministrador', 'Material', 'Texto breve', 'Grupo de artículos', 'Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de pedido', 'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad de imputación']


In [46]:
#renombrar columnas especiicas

columnas_renombrar = {
    'Posición': 'Posición PO',
    'Fecha documento': 'Fecha de PO',
    'Documento compras': 'PO',
    'Posición.1':'Posición KMEX',
    'Doc.comercial': 'SO KMEX'

}
df_base_limpio = df_base_limpio.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df_base_limpio.columns})
#2. agregar columna 'SO AKG' vacía
df_base_limpio['SO AKG'] = ""
print(df_base_limpio.head(3))


           PO  Posición PO     SO KMEX  Posición KMEX Fecha de PO  \
0  4512883075           10   1259948.0             10  2025-01-02   
1  4512885572          110  53868810.0          10150  2025-01-03   
2  4512885594          260  53868829.0          10690  2025-01-03   

  Proveedor/Centro suministrador    Material  \
0           650100     KRONES AG  0907308319   
1          650018     KRONES INC  0903181065   
2          650018     KRONES INC  0904639435   

                                Texto breve Grupo de artículos  \
0  Change of name plates, cable tags and do               VW00   
1    Direct-on-line starter 0.45 - 0.63A/10          EI0050000   
2                             Lamp assembly          EK0200000   

  Tipo de posición Tipo de imputación  Centro  Almacén  Cantidad de pedido  \
0              NaN                  3    8301      1.0                 1.0   
1              NaN                  M    8301      1.0                 2.0   
2              NaN             

CÓMO SE OBTUVO TIPO DE ENVÍO

In [47]:
df_base_limpio.columns

Index(['PO', 'Posición PO', 'SO KMEX', 'Posición KMEX', 'Fecha de PO',
       'Proveedor/Centro suministrador', 'Material', 'Texto breve',
       'Grupo de artículos', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de pedido', 'Unidad medida pedido',
       'Precio neto', 'Moneda', 'Cantidad de imputación', 'SO AKG'],
      dtype='object')

In [49]:
print(df_base_limpio.columns)

Index(['PO', 'Posición PO', 'SO KMEX', 'Posición KMEX', 'Fecha de PO',
       'Proveedor/Centro suministrador', 'Material', 'Texto breve',
       'Grupo de artículos', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de pedido', 'Unidad medida pedido',
       'Precio neto', 'Moneda', 'Cantidad de imputación', 'SO AKG'],
      dtype='object')


Funcion modular VLOOKUP

In [51]:
def vlookup_en_df(
    df_destino: pd.DataFrame, #excel o df al qe se agrega el valor
    df_origen: pd.DataFrame,  #de donde se extrae la informaicon
    columna_busqueda: str,    #Columna en df_destino a buscar en df_origen
    columna_origen_key: str,  #Columna en df_origen para comparar, columna clvae en df orien
    columna_valor: str,       #columna en df_origen de donde traer el vlaor
    columna_nueva: str        #columna en df_destino donde poner el valor traído

) -> pd.DataFrame:

  #Creamos un diccionario {key.valor}
  dic_vlookup= df_origen.set_index(columna_origen_key)[columna_valor].to_dict()
  #Asiganmos el resultado usando map(rápido y eficiente)
  df_destino[columna_nueva] = df_destino[columna_busqueda].map(dic_vlookup)
  return df_destino
